In [ ]:
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization       
from keras.layers import LeakyReLU
from copy import copy, deepcopy
from numpy import load
from sklearn.model_selection import train_test_split
from numpy import zeros
from numpy import ones
from numpy.random import randint
from matplotlib import pyplot

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import PIL
import re
import pandas as pd
from osgeo import gdal
import os
from os import listdir
from numpy import asarray
from tensorflow.keras.backend import clear_session
from numpy import vstack
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from numpy import savez_compressed
from IPython import display

In [ ]:
ROOT_PATH = "data/vrt/s2/rgb/"
DATA_PATH = "/Volumes/X/Data/fusion-s1-s2/"

In [ ]:
def create_crops(image, crop_height, crop_width, stride, num_bands):
    cropped_images = []

    for y in range(0, image.shape[0] - crop_height + 1, stride):
        for x in range(0, image.shape[1] - crop_width + 1, stride):
            cropped_image = image[y:y + crop_height, x:x + crop_width, :num_bands]
            cropped_images.append(cropped_image)

    return np.array(cropped_images)

In [ ]:
def read_and_preprocess_vrt(file_path, crop_height, crop_width, stride, crop=False):
    # Read the image
    img = gdal.Open(file_path)
    bands = img.RasterCount
    rows, cols = img.RasterYSize, img.RasterXSize
    image = np.zeros((rows, cols, bands))

    for band_index in range(bands):
        bandx = img.GetRasterBand(band_index + 1)
        datax = bandx.ReadAsArray()
        image[:, :, band_index] = datax

    # Create crops
    if crop:
        cropped_images = create_crops(image, crop_height, crop_width, stride, bands)
        return cropped_images

    return image

In [ ]:
cloud_free_vrts = [
    # "data/vrt/s2/translate/S2_32VNH_20190227.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20190418.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20190423.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20190831.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20200422.vrt",
    "data/vrt/s2/translate/S2_32VNH_20200601.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20200914.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20211029.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20190418.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20190423.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20190831.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20200323.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20200422.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20200601.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20200914.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20210616.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20210909.vrt"
    # "data/vrt/s2/translate/S2_32VNH_20211029.vrt",
]

cloudy_vrts = [
    # "data/vrt/s2/translate/S2_32VNH_20190113.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20190309.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20190503.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20190513.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20190602.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20190617.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20190811.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20190821.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20190905.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20190915.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20190930.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20191020.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20191114.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20191209.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20200103.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20200113.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20200313.vrt",
    "data/vrt/s2/translate/S2_32VNH_20200611.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20200711.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20200830.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20200904.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20200929.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20201029.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20201113.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20210122.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20210201.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20210303.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20210407.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20210412.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20210422.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20210502.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20210606.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20210914.vrt",
    # "data/vrt/s2/translate/S2_32VNH_20211123.vrt",
]

In [ ]:
import os

for path in cloud_free_vrts:
    if not os.path.isfile(path):
        print(path)
        

In [ ]:
import json

data =  None
with open("data/fmask_stats.json", "r") as f:
    data = json.load(f)

for k, v in data.items():
    t = "no_observation" not in v
    tt = "no_observation" in v and v["no_observation"] < 1
    if t or tt:
        if "cloud" in v and v["cloud"] > 50 and v["cloud"] < 70:
            print(k)

In [ ]:
cloudy_images = [read_and_preprocess_vrt(file_path, 1024, 1024, stride=1024, crop=True) for file_path in cloudy_vrts]
cloud_free_images = [read_and_preprocess_vrt(file_path, 1024, 1024, stride=1024, crop=True) for file_path in cloud_free_vrts]

In [ ]:
cloudy_images=cloudy_images[0]
cloud_free_images=cloud_free_images[0]

In [ ]:
def normalize_images(images):
    # min_value = images.min()
    # max_value = images.max()
    # normalized_images = (images - min_value) / (max_value - min_value)
    normalized_images = images/2000
    return normalized_images

In [ ]:
normalized_cloudy_images = [normalize_images(cloudy_image) for cloudy_image in cloudy_images]
normalized_cloud_free_images = [normalize_images(cloud_free_image) for cloud_free_image in cloud_free_images]

In [ ]:
import matplotlib.pyplot as plt

num_images = 10

for i in range(num_images):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

    # Plot the cloud-free image
    ax1.imshow(normalized_cloud_free_images[i])
    ax1.set_title(f"Cloud-free image {i+1}")
    ax1.axis("off")

    # Plot the corresponding cloudy image
    ax2.imshow(normalized_cloudy_images[i])
    ax2.set_title(f"Cloudy image {i+1}")
    ax2.axis("off")

    # Show the plot
    plt.show()

In [ ]:
# Set the ratios for the dataset split
train_ratio = 0.8
validation_ratio = 0.1
test_ratio = 0.1

# Split the data into training and the remaining sets
X_train, X_remaining, y_train, y_remaining = train_test_split(
    normalized_cloudy_images, normalized_cloud_free_images, test_size=(1 - train_ratio), random_state=42
)

# Calculate the size of the validation and testing sets
remaining_size = validation_ratio + test_ratio
validation_size = validation_ratio / remaining_size
test_size = test_ratio / remaining_size

# Split the remaining data into validation and testing sets
X_val, X_test, y_val, y_test = train_test_split(
    X_remaining, y_remaining, test_size=test_size, random_state=42
)


In [ ]:
X_train = normalized_cloud_free_images
X_test = normalized_cloudy_images

In [ ]:
import matplotlib.pyplot as plt

num_images = 100

for i in range(num_images):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

    # Plot the cloud-free image
    ax1.imshow(X_train[i])
    ax1.set_title(f"Cloud-free image {i+1}")
    ax1.axis("off")

    # Plot the corresponding cloudy image
    ax2.imshow(X_test[i])
    ax2.set_title(f"Cloudy image {i+1}")
    ax2.axis("off")

    # Show the plot
    plt.show()

In [ ]:
#----------training stack-------------#
tem1 = deepcopy(temp_train)
# -------- MY NEW APPROACH -----------------#
# #no skew
# tem3 = deepcopy(temp_train)

# #90 deg clockwise
# tem4 = deepcopy(temp_train)
# tem4 = np.rot90(tem4, axes=(1,0))

# #90 deg anti-clockwise
# tem5 = deepcopy(temp_train)
# tem5 = np.rot90(tem5, axes=(0,1))

# #180 deg 
# #rotate 180 deg clockwise/anti-clockwise
# tem6 = deepcopy(temp_train)
# tem6 = np.rot90(tem6, axes=(1,0))
# tem6 = np.rot90(tem6, axes=(1,0))
# ------------------------------------------#

# ---------- REFERENCE CODE APPROACH --------#
tem3=np.rot90(tem1)
tem4=np.rot90(tem1, 2)
tem5=np.rot90(tem1, 3)
# -------------------------------------------#

#displaying the distinct images of the training stack
fig, a = plt.subplots(1,4, figsize=(16,4))

a[0].imshow(tem1)
a[0].set_title('original')
a[0].axis('off')

a[1].imshow(tem3)
a[1].set_title('-90-deg')
a[1].axis('off')

a[2].imshow(tem4)
a[2].set_title('180-deg')
a[2].axis('off')

a[3].imshow(tem5)
a[3].set_title('+90-deg')
a[3].axis('off')

plt.show()

In [ ]:
#stacking the augmented TRUE images
train_stack = np.stack((tem3, tem4, tem5, tem3, tem4, tem5, tem3, tem4, tem5, tem3), axis=0)
print(train_stack.shape)

#saving the training set on disk
np.save(ROOT_PATH + '/data/train_data_3_10copy', train_stack)

In [ ]:
#----------testing stack-------------#
tem2 = deepcopy(temp_test)
# # -------- MY NEW APPROACH -----------------#
# #no skew
# tem7 = deepcopy(temp_test)

# #90 deg clockwise
# tem8 = deepcopy(temp_test)
# tem8 = np.rot90(tem8, axes=(1,0))

# #90 deg anti-clockwise
# tem9 = deepcopy(temp_test)
# tem9 = np.rot90(tem9, axes=(0,1))

# #180 deg 
# #rotate 180 deg clockwise/anti-clockwise
# tem10 = deepcopy(temp_test)
# tem10 = np.rot90(tem10, axes=(1,0))
# tem10 = np.rot90(tem10, axes=(1,0))
# # ------------------------------------------#

# ---------- REFERENCE CODE APPROACH --------#
tem6=np.rot90(tem2)
tem7=np.rot90(tem2, 2)
tem8=np.rot90(tem2, 3)
# -------------------------------------------#

#displaying the distinct images of the training stack
fig, a = plt.subplots(1,4, figsize=(16,4))


a[0].imshow(tem2)
a[0].set_title('original')
a[0].axis('off')

a[1].imshow(tem6)
a[1].set_title('-90-deg')
a[1].axis('off')

a[2].imshow(tem7)
a[2].set_title('180-deg')
a[2].axis('off')

a[3].imshow(tem8)
a[3].set_title('+90-deg')
a[3].axis('off')

plt.show()

In [ ]:
#stacking the augmented FALSE images
train_stack = np.stack((tem6, tem7, tem8, tem6, tem7, tem8, tem6, tem7, tem8, tem6), axis=0)
print(train_stack.shape)

#saving the testing set on disk
np.save(ROOT_PATH + '/data/test_data_3_10copy', train_stack)

In [ ]:
X_train_np = np.stack(X_train)
y_train_np = np.stack(y_train)
X_val_np = np.stack(X_val)
y_val_np = np.stack(y_val)
X_test_np = np.stack(X_test)
y_test_np = np.stack(y_test)

print(X_test_np.shape)
print(X_train_np.shape)

In [ ]:
#Discriminator Model

def define_discriminator(image_shape):
  '''
  This function portrays the discriminator model based on the image dimensions
  args --> image_shape: (x,y) for 2d; (x,y,d) for 3d
  '''
  init = RandomNormal(stddev=0.02)
  in_src_image = Input(shape=image_shape)
  # target image input
  in_target_image = Input(shape=image_shape)

  merged = Concatenate()([in_src_image, in_target_image])

  # C64
  d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
  d = LeakyReLU(alpha=0.2)(d)

  # C128
  d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
  d = BatchNormalization()(d)
  d = LeakyReLU(alpha=0.2)(d)
  
  # C256
  d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
  d = BatchNormalization()(d)
  d = LeakyReLU(alpha=0.2)(d)
  
  # C512
  d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
  d = BatchNormalization()(d)
  d = LeakyReLU(alpha=0.2)(d)
  d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
  d = BatchNormalization()(d)
  d = LeakyReLU(alpha=0.2)(d)

  # patch output
  d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
  patch_out = Activation('sigmoid')(d)
  
  # define model
  model = Model([in_src_image, in_target_image], patch_out)
  
  # compile model
  opt = Adam(lr=0.0004, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
  
  return model

In [ ]:
#ENCODER BLOCK

def define_encoder_block(layer_in, n_filters, batchnorm=True):
  '''
  This function protrays the architecture of an encoder block
  '''
  # weight initialization
  init = RandomNormal(stddev=0.02)

  #add downsampling layer
  g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)  	# add downsampling layer

  # conditionally add batch normalization
  if batchnorm:
    g = BatchNormalization()(g, training=True)

  #Activating Leaky RelU
  g = LeakyReLU(alpha=0.2)(g) 	
  return g

In [ ]:
# DECODER BLOCK

def decoder_block(layer_in, skip_in, n_filters, dropout=True):
  '''
  This function portrays the architecture of a decoder block
  '''
  #weight initialization
  init = RandomNormal(stddev=0.02)

  #add upsampling layer
  g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in) 

  #add batch normalization
  g = BatchNormalization()(g, training=True)  	

  # conditionally add dropout
  if dropout:
    g = Dropout(0.5)(g, training=True)
    
  # merge with skip connection
  #basically we concatenate the layers produced by (upconvolution) and the original layer in encoder block
  g = Concatenate()([g, skip_in]) 

  # relu activation
  g = Activation('relu')(g)
  return g

In [ ]:
#Generator Model

def define_generator(image_shape=(256,256,3)):
  '''
  This function portrays the generator model based on the image dimensions
  args --> image_shape: (x,y) for 2d; (x,y,d) for 3d
  '''
  # weight initialization
  init = RandomNormal(stddev=0.02)

  # image input
  in_image = Input(shape=image_shape)
  
  # encoder model
  e1 = define_encoder_block(in_image, 64, batchnorm=False)
  e2 = define_encoder_block(e1, 128)
  e3 = define_encoder_block(e2, 256)
  e4 = define_encoder_block(e3, 512)
  e5 = define_encoder_block(e4, 512)
  e6 = define_encoder_block(e5, 512)
  e7 = define_encoder_block(e6, 512)
  
  # bottleneck, no batch norm and relu
  b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
  b = Activation('relu')(b)
  
  # decoder model
  d1 = decoder_block(b, e7, 512)
  d2 = decoder_block(d1, e6, 512)
  d3 = decoder_block(d2, e5, 512)
  d4 = decoder_block(d3, e4, 512, dropout=False)
  d5 = decoder_block(d4, e3, 256, dropout=False)
  d6 = decoder_block(d5, e2, 128, dropout=False)
  d7 = decoder_block(d6, e1, 64, dropout=False)
  
  # output
  g = Conv2DTranspose(3, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
  out_image = Activation('tanh')(g)
  
  # define model
  model = Model(in_image, out_image)
  
  return model

In [ ]:
# Combine generator and Discriminator Model
def define_gan(g_model, d_model, image_shape):
  '''
  This function portrays the proposed GAN architecture
  args--> generator model, discriminator model, image shape 
  '''
  # make weights in the discriminator not trainable
  d_model.trainable = False
  
  # define the source image
  in_src = Input(shape=image_shape)
  
  # connect the source image to the generator input
  gen_out = g_model(in_src)
  
  # connect the source input and generator output to the discriminator input
  dis_out = d_model([in_src, gen_out])
  
  # src image as input, generated image and classification output
  model = Model(in_src, [dis_out, gen_out])
  
  # compile model
  opt = Adam(lr=0.0004, beta_1=0.5)
  model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
  
  return model

In [ ]:
# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples, patch_shape):
  '''
  This functions selects a batch of random samples and returns an image with its target
  args --> image_stack, number of samples, patch shape
  '''
  # unpack dataset
  trainA, trainB = dataset
  
  # choose random instances
  ix = randint(0, trainA.shape[0], n_samples)
  
  # retrieve selected images
  X1, X2 = trainA[ix], trainB[ix]
  
  # generate 'real' class labels (1)
  y = ones((n_samples, patch_shape, patch_shape, 1))
  
  return [X1, X2], y

In [ ]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, patch_shape):
  '''
  This function uses the generator model to generate fake samples 
  args --> generator model, number of samples, patch shape
  '''
  # generate fake instance
  X = g_model.predict(samples)
  # create 'fake' class labels (0)
  y = zeros((len(X), patch_shape, patch_shape, 1))
  return X, y

In [ ]:
#Epoch operations to summarize model performance

def summarize_performance(step, g_model, dataset, n_samples=3):
  '''
  This function generates sample to pass on to the GAN, and creates the plot at 100X epochs with the src, tar and generated image
  args --> epoch step, generator model, dataset, number of samples
  '''
  # select a sample of input images
  [X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
  
  # generate a batch of fake samples
  X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
  
  # scale all pixels from [-1,1] to [0,1]
  X_realA = (X_realA + 1) / 2.0
  X_realB = (X_realB + 1) / 2.0
  X_fakeB = (X_fakeB + 1) / 2.0
  
  # plot real source images
  for i in range(n_samples):
    pyplot.subplot(3, n_samples, 1 + i)
    pyplot.axis('off')
    pyplot.imshow(X_realA[i])
  
  # plot generated target image
  for i in range(n_samples):
    pyplot.subplot(3, n_samples, 1 + n_samples + i)
    pyplot.axis('off')
    pyplot.imshow(X_fakeB[i])
  
  # plot real target image
  for i in range(n_samples):
    pyplot.subplot(3, n_samples, 1 + n_samples*2 + i)
    pyplot.axis('off')
    pyplot.imshow(X_realB[i])
  
  # save plot to file
  filename1 = 'plot_%06d.png' % (step+1)
  pyplot.savefig(filename1)
  pyplot.close()
  
  # save the generator model
  filename2 = 'model_%06d.h5' % (step+1)
  g_model.save(filename2)
  print('>Saved: %s and %s' % (filename1, filename2))

In [ ]:
def train(d_model, g_model, gan_model, dataset, n_epochs=200, n_batch=1):
  '''
  This function is for training the model based on the pixel-to-pixel architecture to achieve image to image tramslation using GAM
  args --> discriminator model, generator model, GAN model, train/test set, number of epochs, number of train batches/batch size
  '''
  # determine the output square shape of the discriminator
  n_patch = d_model.output_shape[1]
  
  # unpack dataset
  trainA, trainB = dataset
  
  # calculate the number of batches per training epoch
  bat_per_epo = int(len(trainA) / n_batch)
  
  # calculate the number of training iterations
  n_steps = bat_per_epo * n_epochs
  
  #log all losses
  d_loss1_log = []
  d_loss2_log = []
  g_loss_log = []

  # manually enumerate epochs
  for i in range(n_steps):
    # select a batch of real samples
    [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
    
    # generate a batch of fake samples
    X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
    
    # update discriminator for real samples
    d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
    d_loss1_log.append(d_loss1)
    
    # update discriminator for generated samples
    d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
    d_loss2_log.append(d_loss2)
    
    # update the generator
    g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
    g_loss_log.append(g_loss)
    
    # summarize performance - plot loss per epoch
    
    plt.clf()
    plt.figure(figsize=(20,12))
    plt.title('Epoch:%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
    plt.xlabel('Epoch', fontsize=16)
    plt.ylabel('Loss', fontsize=16)
    plt.plot(d_loss1_log, 'r-', lw=2, label='d_loss1')
    plt.plot(d_loss2_log, 'b-', lw=1, label='d_loss2')
    plt.plot(g_loss_log, 'g-', lw=1, label='g_loss')
    plt.legend(prop={'size':16})
    display.clear_output(wait=True)
    display.display(plt.gcf())
    print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
    
    # summarize model performance
    # if (i+1) % (bat_per_epo * 10) == 0:
    if (i+1) in [100, 200]:
      plt.savefig('./loss_graph_%06d.jpg' % (i+1), bbox_inches='tight')
      summarize_performance(i, g_model, dataset)

In [ ]:
clear_session()

In [ ]:
#Training Model
dataset=(X_train_np,X_train_np)
print('loaded - ', X_train_np.shape, X_train_np.shape)

# define input shape based on the loaded dataset
image_shape=X_train_np.shape[1:]

# define the models
d_model = define_discriminator(image_shape)
g_model = define_generator(image_shape)

# define the composite model
gan_model = define_gan(g_model, d_model, image_shape)

# train model
train(d_model, g_model, gan_model, dataset, n_epochs=200)

##Step 7: Predicting the Cloud Free Image

In the previouos step, we have trained the model for predef number of epochs and predrf batch size. The model at each 100 epoch has been saved on the disk. From this point on, we can choose not to execute the previous steps as we now hace the option of loding the model straight from the disk and making the prediction. 

> 7.1 Importing packages to have seperate implementation from this step